d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# Bronze to Silver - ETL into a Silver table

We need to perform some transformations on the data to move it from bronze to silver tables.

## Notebook Objective

In this notebook we:
1. Ingest raw data using composable functions
1. Use composable functions to write to the Bronze table
1. Develop the Bronze to Silver Step
   - Extract and transform the raw string to columns
   - Quarantine the bad data
   - Load clean data into the Silver table
1. Update the status of records in the Bronze table

## Step Configuration

In [0]:
#%run ./includes/configuration

## Import Operation Functions

In [0]:
#%run ./includes/main/python/operations

# Functions show in below chunks
# get all packages here
from typing import List
from pyspark.sql.session import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.sql.functions import *



### Display the Files in the Bronze Paths

In [0]:
#display(dbutils.fs.ls(bronzePath))
#dbutils.fs.rm(bronzePath, recurse=True)

## Land More Raw Data

Before we get started with this lab, let's land some more raw data.

In a production setting, we might have data coming in every
hour. Here we are simulating this with the function
`ingest_classic_data`.

😎 Recall that we did this in the notebook `00_ingest_raw`.

**EXERCISE:** Land ten hours using the utility function, `ingest_classic_data`.

In [0]:
# TODO
#ingest_classic_data(hours=10)

## Current Delta Architecture
Next, we demonstrate everything we have built up to this point in our
Delta Architecture.

We do so not with the ad hoc queries as written before, but now with
composable functions included in the file `classic/includes/main/python/operations`.
You should check this file for the correct arguments to use in the next
three steps.

🤔 You can refer to `plus/02_bronze_to_silver` if you are stuck.

### Step 1: Create the `rawDF` DataFrame

**Exercise:** Use the function `read_batch_raw` to ingest the newly arrived
data.

In [0]:
#Function
def read_batch_raw(rawPath: str) -> DataFrame:
    movie_schema = 'movie ARRAY<STRING>'
    movie_data_df = (
    spark.read.format("json").option("multiline","true").schema(movie_schema).load(path = rawPath)
)
    return movie_data_df.withColumn('movie', explode('movie'))
#Function

movie_path = "/FileStore/movie/*.json"
rawDF = read_batch_raw(movie_path)
display(rawDF.limit(5))

movie
"{""Id"":1,""Title"":""Inception"",""Overview"":""Cobb, a skilled thief who commits corporate espionage by infiltrating the subconscious of his targets is offered a chance to regain his old life as payment for a task considered to be impossible: \""inception\"", the implantation of another person's idea into a target's subconscious."",""Tagline"":""Your mind is the scene of the crime."",""Budget"":1.6E8,""Revenue"":8.25532764E8,""ImdbUrl"":""https://www.imdb.com/title/tt1375666"",""TmdbUrl"":""https://www.themoviedb.org/movie/27205"",""PosterUrl"":""https://image.tmdb.org/t/p/w342//9gk7adHYeDvHkCSEqAvQNLV5Uge.jpg"",""BackdropUrl"":""https://image.tmdb.org/t/p/original//s3TBrRGB1iav7gFOCNx3H31MoES.jpg"",""OriginalLanguage"":""en"",""ReleaseDate"":""2010-07-15T00:00:00"",""RunTime"":148,""Price"":9.9,""CreatedDate"":""2021-04-03T16:51:30.1633333"",""UpdatedDate"":null,""UpdatedBy"":null,""CreatedBy"":null,""genres"":[{""id"":1,""name"":""Adventure""},{""id"":6,""name"":""Action""},{""id"":13,""name"":""Science Fiction""}]}"
"{""Id"":2,""Title"":""Interstellar"",""Overview"":""The adventures of a group of explorers who make use of a newly discovered wormhole to surpass the limitations on human space travel and conquer the vast distances involved in an interstellar voyage."",""Tagline"":""Mankind was born on Earth. It was never meant to die here."",""Budget"":1.65E8,""Revenue"":6.75120017E8,""ImdbUrl"":""https://www.imdb.com/title/tt0816692"",""TmdbUrl"":""https://www.themoviedb.org/movie/157336"",""PosterUrl"":""https://image.tmdb.org/t/p/w342//gEU2QniE6E77NI6lCU6MxlNBvIx.jpg"",""BackdropUrl"":""https://image.tmdb.org/t/p/original//xJHokMbljvjADYdit5fK5VQsXEG.jpg"",""OriginalLanguage"":""en"",""ReleaseDate"":""2014-11-05T00:00:00"",""RunTime"":169,""Price"":9.9,""CreatedDate"":""2021-04-03T16:51:30.1633333"",""UpdatedDate"":null,""UpdatedBy"":null,""CreatedBy"":null,""genres"":[{""id"":1,""name"":""Adventure""},{""id"":4,""name"":""Drama""},{""id"":13,""name"":""Science Fiction""}]}"
"{""Id"":3,""Title"":""The Dark Knight"",""Overview"":""Batman raises the stakes in his war on crime. With the help of Lt. Jim Gordon and District Attorney Harvey Dent, Batman sets out to dismantle the remaining criminal organizations that plague the streets. The partnership proves to be effective, but they soon find themselves prey to a reign of chaos unleashed by a rising criminal mastermind known to the terrified citizens of Gotham as the Joker."",""Tagline"":""Why So Serious?"",""Budget"":1.85E8,""Revenue"":1.004558444E9,""ImdbUrl"":""https://www.imdb.com/title/tt0468569"",""TmdbUrl"":""https://www.themoviedb.org/movie/155"",""PosterUrl"":""https://image.tmdb.org/t/p/w342//qJ2tW6WMUDux911r6m7haRef0WH.jpg"",""BackdropUrl"":""https://image.tmdb.org/t/p/original//hkBaDkMWbLaf8B1lsWsKX7Ew3Xq.jpg"",""OriginalLanguage"":""en"",""ReleaseDate"":""2008-07-16T00:00:00"",""RunTime"":152,""Price"":9.9,""CreatedDate"":""2021-04-03T16:51:30.1633333"",""UpdatedDate"":null,""UpdatedBy"":null,""CreatedBy"":null,""genres"":[{""id"":4,""name"":""Drama""},{""id"":6,""name"":""Action""},{""id"":10,""name"":""Thriller""},{""id"":11,""name"":""Crime""}]}"
"{""Id"":4,""Title"":""Deadpool"",""Overview"":""Deadpool tells the origin story of former Special Forces operative turned mercenary Wade Wilson, who after being subjected to a rogue experiment that leaves him with accelerated healing powers, adopts the alter ego Deadpool. Armed with his new abilities and a dark, twisted sense of humor, Deadpool hunts down the man who nearly destroyed his life."",""Tagline"":""Witness the beginning of a happy ending"",""Budget"":5.8E7,""Revenue"":7.831E8,""ImdbUrl"":""https://www.imdb.com/title/tt1431045"",""TmdbUrl"":""https://www.themoviedb.org/movie/293660"",""PosterUrl"":""https://image.tmdb.org/t/p/w342//yGSxMiF0cYuAiyuve5DA6bnWEOI.jpg"",""BackdropUrl"":""https://image.tmdb.org/t/p/original//en971MEXui9diirXlogOrPKmsEn.jpg"",""OriginalLanguage"":""en"",""ReleaseDate"":""201

### Step 2: Transform the Raw Data

**Exercise:** Use the function `transform_raw` to ingest the newly arrived
data.

In [0]:
# Function
def transform_raw(raw: DataFrame) -> DataFrame:
    return raw.select(
        lit("Antra").alias("datasource"),
        current_timestamp().alias("ingesttime"),
        lit("new").alias("status"),
        "movie",
        current_timestamp().cast("date").alias("p_ingestdate"),
    )
# Function
transformedRawDF = transform_raw(rawDF)
display(transformedRawDF.limit(5))


datasource,ingesttime,status,movie,p_ingestdate
Antra,2022-09-02T09:32:20.458+0000,new,"{""Id"":1,""Title"":""Inception"",""Overview"":""Cobb, a skilled thief who commits corporate espionage by infiltrating the subconscious of his targets is offered a chance to regain his old life as payment for a task considered to be impossible: \""inception\"", the implantation of another person's idea into a target's subconscious."",""Tagline"":""Your mind is the scene of the crime."",""Budget"":1.6E8,""Revenue"":8.25532764E8,""ImdbUrl"":""https://www.imdb.com/title/tt1375666"",""TmdbUrl"":""https://www.themoviedb.org/movie/27205"",""PosterUrl"":""https://image.tmdb.org/t/p/w342//9gk7adHYeDvHkCSEqAvQNLV5Uge.jpg"",""BackdropUrl"":""https://image.tmdb.org/t/p/original//s3TBrRGB1iav7gFOCNx3H31MoES.jpg"",""OriginalLanguage"":""en"",""ReleaseDate"":""2010-07-15T00:00:00"",""RunTime"":148,""Price"":9.9,""CreatedDate"":""2021-04-03T16:51:30.1633333"",""UpdatedDate"":null,""UpdatedBy"":null,""CreatedBy"":null,""genres"":[{""id"":1,""name"":""Adventure""},{""id"":6,""name"":""Action""},{""id"":13,""name"":""Science Fiction""}]}",2022-09-02
Antra,2022-09-02T09:32:20.458+0000,new,"{""Id"":2,""Title"":""Interstellar"",""Overview"":""The adventures of a group of explorers who make use of a newly discovered wormhole to surpass the limitations on human space travel and conquer the vast distances involved in an interstellar voyage."",""Tagline"":""Mankind was born on Earth. It was never meant to die here."",""Budget"":1.65E8,""Revenue"":6.75120017E8,""ImdbUrl"":""https://www.imdb.com/title/tt0816692"",""TmdbUrl"":""https://www.themoviedb.org/movie/157336"",""PosterUrl"":""https://image.tmdb.org/t/p/w342//gEU2QniE6E77NI6lCU6MxlNBvIx.jpg"",""BackdropUrl"":""https://image.tmdb.org/t/p/original//xJHokMbljvjADYdit5fK5VQsXEG.jpg"",""OriginalLanguage"":""en"",""ReleaseDate"":""2014-11-05T00:00:00"",""RunTime"":169,""Price"":9.9,""CreatedDate"":""2021-04-03T16:51:30.1633333"",""UpdatedDate"":null,""UpdatedBy"":null,""CreatedBy"":null,""genres"":[{""id"":1,""name"":""Adventure""},{""id"":4,""name"":""Drama""},{""id"":13,""name"":""Science Fiction""}]}",2022-09-02
Antra,2022-09-02T09:32:20.458+0000,new,"{""Id"":3,""Title"":""The Dark Knight"",""Overview"":""Batman raises the stakes in his war on crime. With the help of Lt. Jim Gordon and District Attorney Harvey Dent, Batman sets out to dismantle the remaining criminal organizations that plague the streets. The partnership proves to be effective, but they soon find themselves prey to a reign of chaos unleashed by a rising criminal mastermind known to the terrified citizens of Gotham as the Joker."",""Tagline"":""Why So Serious?"",""Budget"":1.85E8,""Revenue"":1.004558444E9,""ImdbUrl"":""https://www.imdb.com/title/tt0468569"",""TmdbUrl"":""https://www.themoviedb.org/movie/155"",""PosterUrl"":""https://image.tmdb.org/t/p/w342//qJ2tW6WMUDux911r6m7haRef0WH.jpg"",""BackdropUrl"":""https://image.tmdb.org/t/p/original//hkBaDkMWbLaf8B1lsWsKX7Ew3Xq.jpg"",""OriginalLanguage"":""en"",""ReleaseDate"":""2008-07-16T00:00:00"",""RunTime"":152,""Price"":9.9,""CreatedDate"":""2021-04-03T16:51:30.1633333"",""UpdatedDate"":null,""UpdatedBy"":null,""CreatedBy"":null,""genres"":[{""id"":4,""name"":""Drama""},{""id"":6,""name"":""Action""},{""id"":10,""name"":""Thriller""},{""id"":11,""name"":""Crime""}]}",2022-09-02
Antra,2022-09-02T09:32:20.458+0000,new,"{""Id"":4,""Title"":""Deadpool"",""Overview"":""Deadpool tells the origin story of former Special Forces operative turned mercenary Wade Wilson, who after being subjected to a rogue experiment that leaves him with accelerated healing powers, adopts the alter ego Deadpool. Armed with his new abilities and a dark, twisted sense of humor, Deadpool hunts down the man who nearly destroyed his life."",""Tagline"":""Witness the beginning of a happy ending"",""Budget"":5.8E7,""Revenue"":7.831E8,""ImdbUrl"":""https://www.imdb.com/title/tt1431045"",""TmdbUrl"":""https://www.themoviedb.org/movie

## Verify the Schema with an Assertion

The DataFrame `transformedRawDF` should now have the following schema:

```
datasource: string
ingesttime: timestamp
status: string
value: string
p_ingestdate: date
```

In [0]:
transformedRawDF.printSchema()

root
-- datasource: string (nullable = false)
-- ingesttime: timestamp (nullable = false)
-- status: string (nullable = false)
-- movie: string (nullable = true)
-- p_ingestdate: date (nullable = false)

In [0]:
from pyspark.sql.types import *

assert transformedRawDF.schema == StructType(
    [
        StructField("datasource", StringType(), False),
        StructField("ingesttime", TimestampType(), False),
        StructField("status", StringType(), False),
        StructField("movie", StringType(),True),
        StructField("p_ingestdate", DateType(), False),
    ]
),"Schemas do not match"
print("Assertion passed.")



Assertion passed.

### Step 3: Write Batch to a Bronze Table

**Exercise:** Use the function `batch_writer` to ingest the newly arrived
data.

**Note**: you will need to begin the write with the `.save()` method on
your writer.

🤖 **Be sure to partition on `p_ingestdate`**.

In [0]:
# batch_writer Function

def batch_writer(
    dataframe: DataFrame,
    partition_column: str,
    exclude_columns: List = [],
    mode: str = "append",
) -> DataFrame:
    return (
        dataframe.drop(
            *exclude_columns
        )  # This uses Python argument unpacking (https://docs.python.org/3/tutorial/controlflow.html#unpacking-argument-lists)
        .write.format("delta")
        .mode(mode)
        .partitionBy(partition_column)
    )
# Function
# BronzePath
bronzePath = "/FileStore/movieBronze"


rawToBronzeWriter = batch_writer(dataframe=transformedRawDF, partition_column="p_ingestdate")

rawToBronzeWriter.save(bronzePath)

## Purge Raw File Path

Manually purge the raw files that have already been loaded.

In [0]:
#dbutils.fs.rm(bronzePath, recurse=True)
#dbutils.fs.rm(silverPath, recurse=True)

Out[85]: False

## Display the Bronze Table

In [0]:
%sql
SELECT * 
FROM raw_movie_data_clissic_bronze 
limit 5

datasource,ingesttime,status,movie,p_ingestdate
Antra,2022-09-02T09:32:26.945+0000,new,"{""Id"":1,""Title"":""Inception"",""Overview"":""Cobb, a skilled thief who commits corporate espionage by infiltrating the subconscious of his targets is offered a chance to regain his old life as payment for a task considered to be impossible: \""inception\"", the implantation of another person's idea into a target's subconscious."",""Tagline"":""Your mind is the scene of the crime."",""Budget"":1.6E8,""Revenue"":8.25532764E8,""ImdbUrl"":""https://www.imdb.com/title/tt1375666"",""TmdbUrl"":""https://www.themoviedb.org/movie/27205"",""PosterUrl"":""https://image.tmdb.org/t/p/w342//9gk7adHYeDvHkCSEqAvQNLV5Uge.jpg"",""BackdropUrl"":""https://image.tmdb.org/t/p/original//s3TBrRGB1iav7gFOCNx3H31MoES.jpg"",""OriginalLanguage"":""en"",""ReleaseDate"":""2010-07-15T00:00:00"",""RunTime"":148,""Price"":9.9,""CreatedDate"":""2021-04-03T16:51:30.1633333"",""UpdatedDate"":null,""UpdatedBy"":null,""CreatedBy"":null,""genres"":[{""id"":1,""name"":""Adventure""},{""id"":6,""name"":""Action""},{""id"":13,""name"":""Science Fiction""}]}",2022-09-02
Antra,2022-09-02T09:32:26.945+0000,new,"{""Id"":2,""Title"":""Interstellar"",""Overview"":""The adventures of a group of explorers who make use of a newly discovered wormhole to surpass the limitations on human space travel and conquer the vast distances involved in an interstellar voyage."",""Tagline"":""Mankind was born on Earth. It was never meant to die here."",""Budget"":1.65E8,""Revenue"":6.75120017E8,""ImdbUrl"":""https://www.imdb.com/title/tt0816692"",""TmdbUrl"":""https://www.themoviedb.org/movie/157336"",""PosterUrl"":""https://image.tmdb.org/t/p/w342//gEU2QniE6E77NI6lCU6MxlNBvIx.jpg"",""BackdropUrl"":""https://image.tmdb.org/t/p/original//xJHokMbljvjADYdit5fK5VQsXEG.jpg"",""OriginalLanguage"":""en"",""ReleaseDate"":""2014-11-05T00:00:00"",""RunTime"":169,""Price"":9.9,""CreatedDate"":""2021-04-03T16:51:30.1633333"",""UpdatedDate"":null,""UpdatedBy"":null,""CreatedBy"":null,""genres"":[{""id"":1,""name"":""Adventure""},{""id"":4,""name"":""Drama""},{""id"":13,""name"":""Science Fiction""}]}",2022-09-02
Antra,2022-09-02T09:32:26.945+0000,new,"{""Id"":3,""Title"":""The Dark Knight"",""Overview"":""Batman raises the stakes in his war on crime. With the help of Lt. Jim Gordon and District Attorney Harvey Dent, Batman sets out to dismantle the remaining criminal organizations that plague the streets. The partnership proves to be effective, but they soon find themselves prey to a reign of chaos unleashed by a rising criminal mastermind known to the terrified citizens of Gotham as the Joker."",""Tagline"":""Why So Serious?"",""Budget"":1.85E8,""Revenue"":1.004558444E9,""ImdbUrl"":""https://www.imdb.com/title/tt0468569"",""TmdbUrl"":""https://www.themoviedb.org/movie/155"",""PosterUrl"":""https://image.tmdb.org/t/p/w342//qJ2tW6WMUDux911r6m7haRef0WH.jpg"",""BackdropUrl"":""https://image.tmdb.org/t/p/original//hkBaDkMWbLaf8B1lsWsKX7Ew3Xq.jpg"",""OriginalLanguage"":""en"",""ReleaseDate"":""2008-07-16T00:00:00"",""RunTime"":152,""Price"":9.9,""CreatedDate"":""2021-04-03T16:51:30.1633333"",""UpdatedDate"":null,""UpdatedBy"":null,""CreatedBy"":null,""genres"":[{""id"":4,""name"":""Drama""},{""id"":6,""name"":""Action""},{""id"":10,""name"":""Thriller""},{""id"":11,""name"":""Crime""}]}",2022-09-02
Antra,2022-09-02T09:32:26.945+0000,new,"{""Id"":4,""Title"":""Deadpool"",""Overview"":""Deadpool tells the origin story of former Special Forces operative turned mercenary Wade Wilson, who after being subjected to a rogue experiment that leaves him with accelerated healing powers, adopts the alter ego Deadpool. Armed with his new abilities and a dark, twisted sense of humor, Deadpool hunts down the man who nearly destroyed his life."",""Tagline"":""Witness the beginning of a happy ending"",""Budget"":5.8E7,""Revenue"":7.831E8,""ImdbUrl"":""https://www.imdb.com/title/tt1431045"",""TmdbUrl"":""https://www.themoviedb.org/movie

## Bronze to Silver Step

Let's start the Bronze to Silver step.

## Make Notebook Idempotent

In [0]:

silverPath = "/FileStore/silverPath"

## Load New Records from the Bronze Records

**EXERCISE**

Load all records from the Bronze table with a status of `"new"`.

In [0]:
# TODO
bronzeDF = spark.read.table("raw_movie_data_clissic_bronze").filter("status = 'new'")

## Extract the Nested JSON from the Bronze Records

### Step 1: Extract the Nested JSON from the `value` column
**EXERCISE**

Use `pyspark.sql` functions to extract the `"value"` column as a new
column `"nested_json"`.

In [0]:
# TODO

movie_schema = StructType([
      StructField("BackdropUrl",StringType(),True),
      StructField("Budget",DoubleType(),True),
      StructField("CreatedBy",StringType(),True),
      StructField("CreatedDate",StringType(),True),
      StructField("Id",LongType(),True),
      StructField("ImdbUrl",StringType(),True),
      StructField("OriginalLanguage",StringType(),True),
      StructField("Overview",StringType(),True),
      StructField("PosterUrl",StringType(),True),
      StructField("Price",DoubleType(),True),
      StructField("ReleaseDate",StringType(),True),
      StructField("Revenue",DoubleType(),True),
      StructField("RunTime",LongType(),True),
      StructField("Tagline",StringType(),True),
      StructField("Title",StringType(),True),
      StructField("TmdbUrl",StringType(),True),
      StructField("UpdatedBy",StringType(),True),
      StructField("UpdatedDate",StringType(),True),
      StructField("geres",ArrayType(
        StructType([
        StructField("id", LongType(),True),
        StructField("name", StringType(), True)]),True), True)
  ])

bronzeAugmentedDF = bronzeDF.withColumn(
    "nested_json",from_json(col("movie"),movie_schema)
)
display(bronzeAugmentedDF.limit(5))

datasource,ingesttime,status,movie,p_ingestdate,nested_json
Antra,2022-09-02T09:32:26.945+0000,new,"{""Id"":1,""Title"":""Inception"",""Overview"":""Cobb, a skilled thief who commits corporate espionage by infiltrating the subconscious of his targets is offered a chance to regain his old life as payment for a task considered to be impossible: \""inception\"", the implantation of another person's idea into a target's subconscious."",""Tagline"":""Your mind is the scene of the crime."",""Budget"":1.6E8,""Revenue"":8.25532764E8,""ImdbUrl"":""https://www.imdb.com/title/tt1375666"",""TmdbUrl"":""https://www.themoviedb.org/movie/27205"",""PosterUrl"":""https://image.tmdb.org/t/p/w342//9gk7adHYeDvHkCSEqAvQNLV5Uge.jpg"",""BackdropUrl"":""https://image.tmdb.org/t/p/original//s3TBrRGB1iav7gFOCNx3H31MoES.jpg"",""OriginalLanguage"":""en"",""ReleaseDate"":""2010-07-15T00:00:00"",""RunTime"":148,""Price"":9.9,""CreatedDate"":""2021-04-03T16:51:30.1633333"",""UpdatedDate"":null,""UpdatedBy"":null,""CreatedBy"":null,""genres"":[{""id"":1,""name"":""Adventure""},{""id"":6,""name"":""Action""},{""id"":13,""name"":""Science Fiction""}]}",2022-09-02,"List(https://image.tmdb.org/t/p/original//s3TBrRGB1iav7gFOCNx3H31MoES.jpg, 1.6E8, null, 2021-04-03T16:51:30.1633333, 1, https://www.imdb.com/title/tt1375666, en, Cobb, a skilled thief who commits corporate espionage by infiltrating the subconscious of his targets is offered a chance to regain his old life as payment for a task considered to be impossible: ""inception"", the implantation of another person's idea into a target's subconscious., https://image.tmdb.org/t/p/w342//9gk7adHYeDvHkCSEqAvQNLV5Uge.jpg, 9.9, 2010-07-15T00:00:00, 8.25532764E8, 148, Your mind is the scene of the crime., Inception, https://www.themoviedb.org/movie/27205, null, null, null)"
Antra,2022-09-02T09:32:26.945+0000,new,"{""Id"":2,""Title"":""Interstellar"",""Overview"":""The adventures of a group of explorers who make use of a newly discovered wormhole to surpass the limitations on human space travel and conquer the vast distances involved in an interstellar voyage."",""Tagline"":""Mankind was born on Earth. It was never meant to die here."",""Budget"":1.65E8,""Revenue"":6.75120017E8,""ImdbUrl"":""https://www.imdb.com/title/tt0816692"",""TmdbUrl"":""https://www.themoviedb.org/movie/157336"",""PosterUrl"":""https://image.tmdb.org/t/p/w342//gEU2QniE6E77NI6lCU6MxlNBvIx.jpg"",""BackdropUrl"":""https://image.tmdb.org/t/p/original//xJHokMbljvjADYdit5fK5VQsXEG.jpg"",""OriginalLanguage"":""en"",""ReleaseDate"":""2014-11-05T00:00:00"",""RunTime"":169,""Price"":9.9,""CreatedDate"":""2021-04-03T16:51:30.1633333"",""UpdatedDate"":null,""UpdatedBy"":null,""CreatedBy"":null,""genres"":[{""id"":1,""name"":""Adventure""},{""id"":4,""name"":""Drama""},{""id"":13,""name"":""Science Fiction""}]}",2022-09-02,"List(https://image.tmdb.org/t/p/original//xJHokMbljvjADYdit5fK5VQsXEG.jpg, 1.65E8, null, 2021-04-03T16:51:30.1633333, 2, https://www.imdb.com/title/tt0816692, en, The adventures of a group of explorers who make use of a newly discovered wormhole to surpass the limitations on human space travel and conquer the vast distances involved in an interstellar voyage., https://image.tmdb.org/t/p/w342//gEU2QniE6E77NI6lCU6MxlNBvIx.jpg, 9.9, 2014-11-05T00:00:00, 6.75120017E8, 169, Mankind was born on Earth. It was never meant to die here., Interstellar, https://www.themoviedb.org/movie/157336, null, null, null)"
Antra,2022-09-02T09:32:26.945+0000,new,"{""Id"":3,""Title"":""The Dark Knight"",""Overview"":""Batman raises the stakes in his war on crime. With the help of Lt. Jim Gordon and District Attorney Harvey Dent, Batman sets out to dismantle the remaining criminal organizations that plague the streets. The partnership proves to be effective, but they soon find themselves prey to a reign of chaos unleashed by a rising criminal mastermind known to the terrified citizens of Gotham as the Joker."",""Tagline"":""Why So Serious?"",""Budget"":1.85E8,""Rev

### Step 2: Create the Silver DataFrame by Unpacking the `nested_json` Column

Unpacking a JSON column means to flatten the JSON and include each top level attribute
as its own column.

🚨 **IMPORTANT** Be sure to include the `"value"` column in the Silver DataFrame
because we will later use it as a unique reference to each record in the
Bronze table

In [0]:
#
silver_movie = bronzeAugmentedDF.select("movie", "nested_json.*")
silver_movie.printSchema()

root
-- movie: string (nullable = true)
-- BackdropUrl: string (nullable = true)
-- Budget: double (nullable = true)
-- CreatedBy: string (nullable = true)
-- CreatedDate: string (nullable = true)
-- Id: long (nullable = true)
-- ImdbUrl: string (nullable = true)
-- OriginalLanguage: string (nullable = true)
-- Overview: string (nullable = true)
-- PosterUrl: string (nullable = true)
-- Price: double (nullable = true)
-- ReleaseDate: string (nullable = true)
-- Revenue: double (nullable = true)
-- RunTime: long (nullable = true)
-- Tagline: string (nullable = true)
-- Title: string (nullable = true)
-- TmdbUrl: string (nullable = true)
-- UpdatedBy: string (nullable = true)
-- UpdatedDate: string (nullable = true)
-- geres: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- id: long (nullable = true)
 | |-- name: string (nullable = true)

## Verify the Schema with an Assertion

The DataFrame `silver_health_tracker` should now have the following schema:

```
value: string
time: timestamp
name: string
device_id: string
steps: integer
day: integer
month: integer
hour: integer
```

💪🏼 Remember, the function `_parse_datatype_string` converts a DDL format schema string into a Spark schema.

In [0]:
from pyspark.sql.types import _parse_datatype_string

assert silver_movie.schema == StructType(
  [
      StructField("movie",StringType(),True),
      StructField("BackdropUrl",StringType(),True),
      StructField("Budget",DoubleType(),True),
      StructField("CreatedBy",StringType(),True),
      StructField("CreatedDate",StringType(),True),
      StructField("Id",LongType(),True),
      StructField("ImdbUrl",StringType(),True),
      StructField("OriginalLanguage",StringType(),True),
      StructField("Overview",StringType(),True),
      StructField("PosterUrl",StringType(),True),
      StructField("Price",DoubleType(),True),
      StructField("ReleaseDate",StringType(),True),
      StructField("Revenue",DoubleType(),True),
      StructField("RunTime",LongType(),True),
      StructField("Tagline",StringType(),True),
      StructField("Title",StringType(),True),
      StructField("TmdbUrl",StringType(),True),
      StructField("UpdatedBy",StringType(),True),
      StructField("UpdatedDate",StringType(),True),
      StructField("geres",ArrayType(
        StructType([
        StructField("id", LongType(),True),
        StructField("name", StringType(), True)]),True), True)
  ]), "Not passed"
print("Assertion passed.")

Assertion passed.

## Transform the Data

1. Create a column `p_eventdate DATE` from the column `time`.
1. Rename the column `time` to `eventtime`.
1. Cast the `device_id` as an integer.
1. Include only the following columns in this order:
   1. `value`
   1. `device_id`
   1. `steps`
   1. `eventtime`
   1. `name`
   1. `p_eventdate`

💪🏼 Remember that we name the new column `p_eventdate` to indicate
that we are partitioning on this column.

🕵🏽‍♀️ Remember that we are keeping the `value` as a unique reference to values
in the Bronze table.

In [0]:
# TODO

silver_movie = silver_movie.select(
     'movie',
     'Id',
     'Title',
     'Overview',
     'OriginalLanguage',
     'Price',
     'ReleaseDate',
     year(col('ReleaseDate').cast('date')).alias('p_ReleaseYear'),
     'Budget',
     'Revenue',
     'RunTime',
     'Tagline',
     'geres',
     'CreatedBy',
     'CreatedDate',
     'UpdatedBy',
     'UpdatedDate',
     'ImdbUrl',
     'TmdbUrl',
     'PosterUrl',
     'BackdropUrl'
)

## Quarantine the Bad Data

Recall that at step, `00_ingest_raw`, we identified that some records were coming in
with device_ids passed as uuid strings instead of string-encoded integers.
Our Silver table stores device_ids as integers so clearly there is an issue
with the incoming data.

In order to properly handle this data quality issue, we will quarantine
the bad records for later processing.

Check for records that have nulls - compare the output of the following two cells

In [0]:
silver_movie.count()
#silver_movie.dropDuplicates().printSchema()

Out[104]: 9995

In [0]:
silver_movie.na.drop().count()

Out[103]: 0

### Split the Silver DataFrame

In [0]:
silver_movie_clean = silver_movie.filter("RunTime >= 0 and Budget >= 1000000")
silver_movie_quarantine = silver_movie.filter("RunTime < 0 or Budget < 1000000 or Budget is NULL")

In [0]:
silver_movie_clean.count()
silver_movie_quarantine.count()

Out[121]: 4488

### Display the Quarantined Records

In [0]:
display(silver_movie_quarantine.limit(5))

movie,Id,Title,Overview,OriginalLanguage,Price,ReleaseDate,p_ReleaseYear,Budget,Revenue,RunTime,Tagline,geres,CreatedBy,CreatedDate,UpdatedBy,UpdatedDate,ImdbUrl,TmdbUrl,PosterUrl,BackdropUrl
"{""Id"":323,""Title"":""Psycho"",""Overview"":""When larcenous real estate clerk Marion Crane goes on the lam with a wad of cash and hopes of starting a new life, she ends up at the notorious Bates Motel, where manager Norman Bates cares for his housebound mother."",""Tagline"":""A new—and altogether different—screen excitement!"",""Budget"":806947.0,""Revenue"":3.2E7,""ImdbUrl"":""https://www.imdb.com/title/tt0054215"",""TmdbUrl"":""https://www.themoviedb.org/movie/539"",""PosterUrl"":""https://image.tmdb.org/t/p/w342//nR4LD4ZJg2n6LZQpcOrLFdMq0cD.jpg"",""BackdropUrl"":""https://image.tmdb.org/t/p/original//kZGaVeXSkkvrpMYvD97sxHj291k.jpg"",""OriginalLanguage"":""en"",""ReleaseDate"":""1960-06-22T00:00:00"",""RunTime"":109,""Price"":9.9,""CreatedDate"":""2021-04-03T16:51:30.2200000"",""UpdatedDate"":null,""UpdatedBy"":null,""CreatedBy"":null,""genres"":[{""id"":4,""name"":""Drama""},{""id"":5,""name"":""Horror""},{""id"":10,""name"":""Thriller""}]}",323,Psycho,"When larcenous real estate clerk Marion Crane goes on the lam with a wad of cash and hopes of starting a new life, she ends up at the notorious Bates Motel, where manager Norman Bates cares for his housebound mother.",en,9.9,1960-06-22T00:00:00,1960,806947.0,3.2E7,109,A new—and altogether different—screen excitement!,null,null,2021-04-03T16:51:30.2200000,null,null,https://www.imdb.com/title/tt0054215,https://www.themoviedb.org/movie/539,https://image.tmdb.org/t/p/w342//nR4LD4ZJg2n6LZQpcOrLFdMq0cD.jpg,https://image.tmdb.org/t/p/original//kZGaVeXSkkvrpMYvD97sxHj291k.jpg
"{""Id"":335,""Title"":""To All the Boys I've Loved Before"",""Overview"":""Lara Jean's love life goes from imaginary to out of control when her secret letters to every boy she's ever fallen for are mysteriously mailed out."",""Tagline"":""The Letters Are Out."",""Budget"":0.0,""Revenue"":0.0,""ImdbUrl"":""https://www.imdb.com/title/tt3846674"",""TmdbUrl"":""https://www.themoviedb.org/movie/466282"",""PosterUrl"":""https://image.tmdb.org/t/p/w342//hKHZhUbIyUAjcSrqJThFGYIR6kI.jpg"",""BackdropUrl"":""https://image.tmdb.org/t/p/original//hBHxBOGQBTMX3bDmqKoAgniZ9hE.jpg"",""OriginalLanguage"":""en"",""ReleaseDate"":""2018-08-16T00:00:00"",""RunTime"":100,""Price"":9.9,""CreatedDate"":""2021-04-03T16:51:30.2233333"",""UpdatedDate"":null,""UpdatedBy"":null,""CreatedBy"":null,""genres"":[{""id"":7,""name"":""Comedy""},{""id"":16,""name"":""Romance""}]}",335,To All the Boys I've Loved Before,Lara Jean's love life goes from imaginary to out of control when her secret letters to every boy she's ever fallen for are mysteriously mailed out.,en,9.9,2018-08-16T00:00:00,2018,0.0,0.0,100,The Letters Are Out.,null,null,2021-04-03T16:51:30.2233333,null,null,https://www.imdb.com/title/tt3846674,https://www.themoviedb.org/movie/466282,https://image.tmdb.org/t/p/w342//hKHZhUbIyUAjcSrqJThFGYIR6kI.jpg,https://image.tmdb.org/t/p/original//hBHxBOGQBTMX3bDmqKoAgniZ9hE.jpg
"{""Id"":429,""Title"":""The Kissing Booth"",""Overview"":""When teenager Elle's first kiss leads to a forbidden romance with the hottest boy in high school, she risks her relationship with her best friend."",""Tagline"":""She can tell her best friend anything, except this one thing"",""Budget"":0.0,""Revenue"":0.0,""ImdbUrl"":""https://www.imdb.com/title/tt3799232"",""TmdbUrl"":""https://www.themoviedb.org/movie/454983"",""PosterUrl"":""https://image.tmdb.org/t/p/w342//7Dktk2ST6aL8h9Oe5rpk903VLhx.jpg"",""BackdropUrl"":""https://image.tmdb.org/t/p/original//itiz2OBK4ns6XT0ufXtusojmMt9.jpg"",""OriginalLanguage"":""en"",""ReleaseDate"":""2018-05-11T00:00:00"",""RunTime"":105,""Price"":9.9,""CreatedDate"":""2021-04-03T16:51:30.2366667"",""UpdatedDate"":null,""UpdatedBy"":null,""CreatedBy"":null,""genres"":[{""id"":7,""name"":""Comedy""},{""id"":16,""name"":""Romance""}]}",429,T

## WRITE Clean Batch to a Silver Table

**EXERCISE:** Batch write `silver_health_tracker_clean` to the Silver table path, `silverPath`.

1. Use format, `"delta"`
1. Use mode `"append"`.
1. Do **NOT** include the `value` column.
1. Partition by `"p_eventdate"`.

In [0]:
# TODO
(
    silver_movie_clean.select(
     'Id',
     'Title',
     'Overview',
     'OriginalLanguage',
     'Price',
     'ReleaseDate',
     'Budget',
     'Revenue',
     'RunTime',
     'Tagline',
     'geres',
     'CreatedBy',
     'CreatedDate',
     'UpdatedBy',
     'UpdatedDate',
     'ImdbUrl',
     'TmdbUrl',
     'PosterUrl',
     'BackdropUrl',
     'p_ReleaseYear'
    )
    .write.format("delta")
    .mode("append")
    .partitionBy("p_ReleaseYear")
    .save(silverPath)
)

In [0]:
spark.sql(
    """
DROP TABLE IF EXISTS silver_movie_silver
"""
)

spark.sql(
    f"""
CREATE TABLE silver_movie_silver
USING DELTA
LOCATION "{silverPath}"
"""
)

Out[124]: DataFrame[]

In [0]:
%sql
SELECT *
FROM silver_movie_silver
LIMIT 5;

Id,Title,Overview,OriginalLanguage,Price,ReleaseDate,Budget,Revenue,RunTime,Tagline,geres,CreatedBy,CreatedDate,UpdatedBy,UpdatedDate,ImdbUrl,TmdbUrl,PosterUrl,BackdropUrl,p_ReleaseYear
461,Snow White and the Seven Dwarfs,"A beautiful girl, Snow White, takes refuge in the forest in the house of seven dwarfs to hide from her stepmother, the wicked Queen. The Queen is jealous because she wants to be known as ""the fairest in the land,"" and Snow White's beauty surpasses her own.",en,9.9,1937-12-21T00:00:00,1488423.0,1.84925486E8,83,"The Happiest, Dopiest, Grumpiest, Sneeziest movie of the year.",null,null,2021-04-03T16:51:30.2433333,null,null,https://www.imdb.com/title/tt0029583,https://www.themoviedb.org/movie/408,https://image.tmdb.org/t/p/w342//4UvFMz8Ys0TaTTxma5e2tKj8u61.jpg,https://image.tmdb.org/t/p/original//pmZiChyJs5Y0tz7sMAiZNNIo9z4.jpg,1937
1246,Modern Times,The Tramp struggles to live in modern industrial society with the help of a young homeless woman.,en,9.9,1936-02-11T00:00:00,1500000.0,8500000.0,87,"He stands alone as the greatest entertainer of modern times! No one on earth can make you laugh as heartily or touch your heart as deeply...the whole world laughs, cries and thrills to his priceless genius!",null,null,2021-04-03T16:51:30.4200000,null,null,https://www.imdb.com/title/tt0027977,https://www.themoviedb.org/movie/3082,https://image.tmdb.org/t/p/w342//7uoiKOEjxBBW0AgDGQWrlfGQ90w.jpg,https://image.tmdb.org/t/p/original//qFWnaG1fVIloWZho6I0td43MpRD.jpg,1936
747,The Wizard of Oz,"Young Dorothy finds herself in a magical world where she makes friends with a lion, a scarecrow and a tin man as they make their way along the yellow brick road to talk with the Wizard and ask for the things they miss most in their lives. The Wicked Witch of the West is the only thing that could stop them.",en,9.9,1939-08-15T00:00:00,2777000.0,3.3754967E7,102,"We're off to see the Wizard, the wonderful Wizard of Oz!",null,null,2021-04-03T16:51:30.2833333,null,null,https://www.imdb.com/title/tt0032138,https://www.themoviedb.org/movie/630,https://image.tmdb.org/t/p/w342//pfAZFD7I2hxW9HCChTuAzsdE6UX.jpg,https://image.tmdb.org/t/p/original//qAvou7F5P4VcIR72JzzrnKEQSN3.jpg,1939
1094,Gone with the Wind,"The spoiled daughter of a well-to-do plantation owner is forced to use every means at her disposal to claw her way out of poverty, following Maj. Gen. William Sherman's destructive ""March to the Sea,” during the American Civil War.",en,9.9,1939-12-15T00:00:00,4000000.0,4.02352579E8,233,The greatest romance of all time!,null,null,2021-04-03T16:51:30.3966667,null,null,https://www.imdb.com/title/tt0031381,https://www.themoviedb.org/movie/770,https://image.tmdb.org/t/p/w342//s0S2KXGRHxOYu8zCcV5p5sczw9d.jpg,https://image.tmdb.org/t/p/original//ft8WRF2xqEwwGWa59naDUybKTAx.jpg,1939
2088,City Lights,"In this sound-era silent film, a tramp falls in love with a beautiful blind flower seller.",en,9.9,1931-02-01T00:00:00,1500000.0,19181.0,87,True Blind Love,null,null,2021-04-03T16:51:30.5900000,null,null,https://www.imdb.com/title/tt0021749,https://www.themoviedb.org/movie/901,https://image.tmdb.org/t/p/w342//bXNvzjULc9jrOVhGfjcc64uKZmZ.jpg,https://image.tmdb.org/t/p/original//ps82zgSDimvlYps6otcMYshrVyS.jpg,1931


## Update Bronze table to Reflect the Loads

**EXERCISE:** Update the records in the Bronze table to reflect updates.

### Step 1: Update Clean records
Clean records that have been loaded into the Silver table and should have
   their Bronze table `status` updated to `"loaded"`.

💃🏽 **Hint** You are matching the `value` column in your clean Silver DataFrame
to the `value` column in the Bronze table.

In [0]:
#silver_movie_clean.dropDuplicates().count()

Out[125]: 5507

In [0]:
# TODO
from delta.tables import DeltaTable

bronzeTable = DeltaTable.forPath(spark, bronzePath)
silverAugmented = (
    silver_movie_clean
    .withColumn("status", lit("loaded"))
)

update_match = "bronze.movie = clean.movie"
update = {"status": "clean.status"}

(
  bronzeTable.alias("bronze")
  .merge(silverAugmented.dropDuplicates().alias("clean"), update_match)
  .whenMatchedUpdate(set=update)
  .execute()
)

**EXERCISE:** Update the records in the Bronze table to reflect updates.

### Step 2: Update Quarantined records
Quarantined records should have their Bronze table `status` updated to `"quarantined"`.

🕺🏻 **Hint** You are matching the `value` column in your quarantine Silver
DataFrame to the `value` column in the Bronze table.

In [0]:
# TODO
silverAugmented = (
  silver_movie_quarantine
  .withColumn("status", lit("quarantined"))
)

update_match = "bronze.movie = quarantine.movie"
update = {"status": "quarantine.status"}

(
  bronzeTable.alias("bronze")
  .merge(silverAugmented.dropDuplicates().alias("quarantine"), update_match)
  .whenMatchedUpdate(set=update)
  .execute()
)


### Update Quarantined records
#### Step 1: loading quarantined dataframe
Updata all 'Quarantined' records to correct data.

In [0]:
bronzeQuarantinedDF = spark.read.table("raw_movie_data_clissic_bronze").filter(
    "status = 'quarantined'"
)
display(bronzeQuarantinedDF.limit(5))

datasource,ingesttime,status,movie,p_ingestdate
Antra,2022-09-02T09:32:26.945+0000,quarantined,"{""Id"":323,""Title"":""Psycho"",""Overview"":""When larcenous real estate clerk Marion Crane goes on the lam with a wad of cash and hopes of starting a new life, she ends up at the notorious Bates Motel, where manager Norman Bates cares for his housebound mother."",""Tagline"":""A new—and altogether different—screen excitement!"",""Budget"":806947.0,""Revenue"":3.2E7,""ImdbUrl"":""https://www.imdb.com/title/tt0054215"",""TmdbUrl"":""https://www.themoviedb.org/movie/539"",""PosterUrl"":""https://image.tmdb.org/t/p/w342//nR4LD4ZJg2n6LZQpcOrLFdMq0cD.jpg"",""BackdropUrl"":""https://image.tmdb.org/t/p/original//kZGaVeXSkkvrpMYvD97sxHj291k.jpg"",""OriginalLanguage"":""en"",""ReleaseDate"":""1960-06-22T00:00:00"",""RunTime"":109,""Price"":9.9,""CreatedDate"":""2021-04-03T16:51:30.2200000"",""UpdatedDate"":null,""UpdatedBy"":null,""CreatedBy"":null,""genres"":[{""id"":4,""name"":""Drama""},{""id"":5,""name"":""Horror""},{""id"":10,""name"":""Thriller""}]}",2022-09-02
Antra,2022-09-02T09:32:26.945+0000,quarantined,"{""Id"":335,""Title"":""To All the Boys I've Loved Before"",""Overview"":""Lara Jean's love life goes from imaginary to out of control when her secret letters to every boy she's ever fallen for are mysteriously mailed out."",""Tagline"":""The Letters Are Out."",""Budget"":0.0,""Revenue"":0.0,""ImdbUrl"":""https://www.imdb.com/title/tt3846674"",""TmdbUrl"":""https://www.themoviedb.org/movie/466282"",""PosterUrl"":""https://image.tmdb.org/t/p/w342//hKHZhUbIyUAjcSrqJThFGYIR6kI.jpg"",""BackdropUrl"":""https://image.tmdb.org/t/p/original//hBHxBOGQBTMX3bDmqKoAgniZ9hE.jpg"",""OriginalLanguage"":""en"",""ReleaseDate"":""2018-08-16T00:00:00"",""RunTime"":100,""Price"":9.9,""CreatedDate"":""2021-04-03T16:51:30.2233333"",""UpdatedDate"":null,""UpdatedBy"":null,""CreatedBy"":null,""genres"":[{""id"":7,""name"":""Comedy""},{""id"":16,""name"":""Romance""}]}",2022-09-02
Antra,2022-09-02T09:32:26.945+0000,quarantined,"{""Id"":429,""Title"":""The Kissing Booth"",""Overview"":""When teenager Elle's first kiss leads to a forbidden romance with the hottest boy in high school, she risks her relationship with her best friend."",""Tagline"":""She can tell her best friend anything, except this one thing"",""Budget"":0.0,""Revenue"":0.0,""ImdbUrl"":""https://www.imdb.com/title/tt3799232"",""TmdbUrl"":""https://www.themoviedb.org/movie/454983"",""PosterUrl"":""https://image.tmdb.org/t/p/w342//7Dktk2ST6aL8h9Oe5rpk903VLhx.jpg"",""BackdropUrl"":""https://image.tmdb.org/t/p/original//itiz2OBK4ns6XT0ufXtusojmMt9.jpg"",""OriginalLanguage"":""en"",""ReleaseDate"":""2018-05-11T00:00:00"",""RunTime"":105,""Price"":9.9,""CreatedDate"":""2021-04-03T16:51:30.2366667"",""UpdatedDate"":null,""UpdatedBy"":null,""CreatedBy"":null,""genres"":[{""id"":7,""name"":""Comedy""},{""id"":16,""name"":""Romance""}]}",2022-09-02
Antra,2022-09-02T09:32:26.945+0000,quarantined,"{""Id"":494,""Title"":""12 Angry Men"",""Overview"":""The defense and the prosecution have rested and the jury is filing into the jury room to decide if a young Spanish-American is guilty or innocent of murdering his father. What begins as an open and shut case soon becomes a mini-drama of each of the jurors' prejudices and preconceptions about the trial, the accused, and each other."",""Tagline"":""Life is in their hands. Death is on their minds."",""Budget"":350000.0,""Revenue"":1000000.0,""ImdbUrl"":""https://www.imdb.com/title/tt0050083"",""TmdbUrl"":""https://www.themoviedb.org/movie/389"",""PosterUrl"":""https://image.tmdb.org/t/p/w342//wh0f80G6GZvYBNiYmvqFngt3IYq.jpg"",""BackdropUrl"":""https://image.tmdb.org/t/p/original//qqHQsStV6exghCM7zbObuYBiYxw.jpg"",""OriginalLanguage"":""en"",""ReleaseDate"":""1957-04-10T00:00:00"",""RunTime"":97,""Price"":9.9,""CreatedDate"":""2021-04-03T16:51:30.2466667"",""UpdatedDate"":null,""UpdatedBy"":null,""CreatedBy"":null,""genres"":[{""id"":4,""name"":""Drama""}]}",2

In [0]:

#Function
def transform_bronze(bronze: DataFrame) -> DataFrame:

    movie_schema = StructType([
      StructField("BackdropUrl",StringType(),True),
      StructField("Budget",DoubleType(),True),
      StructField("CreatedBy",StringType(),True),
      StructField("CreatedDate",StringType(),True),
      StructField("Id",LongType(),True),
      StructField("ImdbUrl",StringType(),True),
      StructField("OriginalLanguage",StringType(),True),
      StructField("Overview",StringType(),True),
      StructField("PosterUrl",StringType(),True),
      StructField("Price",DoubleType(),True),
      StructField("ReleaseDate",StringType(),True),
      StructField("Revenue",DoubleType(),True),
      StructField("RunTime",LongType(),True),
      StructField("Tagline",StringType(),True),
      StructField("Title",StringType(),True),
      StructField("TmdbUrl",StringType(),True),
      StructField("UpdatedBy",StringType(),True),
      StructField("UpdatedDate",StringType(),True),
      StructField("geres",ArrayType(
        StructType([
        StructField("id", LongType(),True),
        StructField("name", StringType(), True)]),True), True)
    ])

    bronzeAugmentedDF = bronze.withColumn(
        "nested_json", from_json(col("movie"), movie_schema)
    )

    silver_movie = bronzeAugmentedDF.select("movie", "nested_json.*")

    silver_movie = silver_movie.select(
          'movie',
          'Id',
          'Title',
          'Overview',
          'OriginalLanguage',
          'Price',
          'ReleaseDate',
          year(col('ReleaseDate').cast('date')).alias('p_ReleaseYear'),
          'Budget',
          'Revenue',
          'RunTime',
          'Tagline',
          'geres',
          'CreatedBy',
          'CreatedDate',
          'UpdatedBy',
          'UpdatedDate',
          'ImdbUrl',
          'TmdbUrl',
          'PosterUrl',
          'BackdropUrl'
        )
    return silver_movie

bronzeQuarTransDF = transform_bronze(bronzeQuarantinedDF).alias(
    "quarantine"
)
display(bronzeQuarTransDF.limit(5))




movie,Id,Title,Overview,OriginalLanguage,Price,ReleaseDate,p_ReleaseYear,Budget,Revenue,RunTime,Tagline,geres,CreatedBy,CreatedDate,UpdatedBy,UpdatedDate,ImdbUrl,TmdbUrl,PosterUrl,BackdropUrl
"{""Id"":323,""Title"":""Psycho"",""Overview"":""When larcenous real estate clerk Marion Crane goes on the lam with a wad of cash and hopes of starting a new life, she ends up at the notorious Bates Motel, where manager Norman Bates cares for his housebound mother."",""Tagline"":""A new—and altogether different—screen excitement!"",""Budget"":806947.0,""Revenue"":3.2E7,""ImdbUrl"":""https://www.imdb.com/title/tt0054215"",""TmdbUrl"":""https://www.themoviedb.org/movie/539"",""PosterUrl"":""https://image.tmdb.org/t/p/w342//nR4LD4ZJg2n6LZQpcOrLFdMq0cD.jpg"",""BackdropUrl"":""https://image.tmdb.org/t/p/original//kZGaVeXSkkvrpMYvD97sxHj291k.jpg"",""OriginalLanguage"":""en"",""ReleaseDate"":""1960-06-22T00:00:00"",""RunTime"":109,""Price"":9.9,""CreatedDate"":""2021-04-03T16:51:30.2200000"",""UpdatedDate"":null,""UpdatedBy"":null,""CreatedBy"":null,""genres"":[{""id"":4,""name"":""Drama""},{""id"":5,""name"":""Horror""},{""id"":10,""name"":""Thriller""}]}",323,Psycho,"When larcenous real estate clerk Marion Crane goes on the lam with a wad of cash and hopes of starting a new life, she ends up at the notorious Bates Motel, where manager Norman Bates cares for his housebound mother.",en,9.9,1960-06-22T00:00:00,1960,806947.0,3.2E7,109,A new—and altogether different—screen excitement!,null,null,2021-04-03T16:51:30.2200000,null,null,https://www.imdb.com/title/tt0054215,https://www.themoviedb.org/movie/539,https://image.tmdb.org/t/p/w342//nR4LD4ZJg2n6LZQpcOrLFdMq0cD.jpg,https://image.tmdb.org/t/p/original//kZGaVeXSkkvrpMYvD97sxHj291k.jpg
"{""Id"":335,""Title"":""To All the Boys I've Loved Before"",""Overview"":""Lara Jean's love life goes from imaginary to out of control when her secret letters to every boy she's ever fallen for are mysteriously mailed out."",""Tagline"":""The Letters Are Out."",""Budget"":0.0,""Revenue"":0.0,""ImdbUrl"":""https://www.imdb.com/title/tt3846674"",""TmdbUrl"":""https://www.themoviedb.org/movie/466282"",""PosterUrl"":""https://image.tmdb.org/t/p/w342//hKHZhUbIyUAjcSrqJThFGYIR6kI.jpg"",""BackdropUrl"":""https://image.tmdb.org/t/p/original//hBHxBOGQBTMX3bDmqKoAgniZ9hE.jpg"",""OriginalLanguage"":""en"",""ReleaseDate"":""2018-08-16T00:00:00"",""RunTime"":100,""Price"":9.9,""CreatedDate"":""2021-04-03T16:51:30.2233333"",""UpdatedDate"":null,""UpdatedBy"":null,""CreatedBy"":null,""genres"":[{""id"":7,""name"":""Comedy""},{""id"":16,""name"":""Romance""}]}",335,To All the Boys I've Loved Before,Lara Jean's love life goes from imaginary to out of control when her secret letters to every boy she's ever fallen for are mysteriously mailed out.,en,9.9,2018-08-16T00:00:00,2018,0.0,0.0,100,The Letters Are Out.,null,null,2021-04-03T16:51:30.2233333,null,null,https://www.imdb.com/title/tt3846674,https://www.themoviedb.org/movie/466282,https://image.tmdb.org/t/p/w342//hKHZhUbIyUAjcSrqJThFGYIR6kI.jpg,https://image.tmdb.org/t/p/original//hBHxBOGQBTMX3bDmqKoAgniZ9hE.jpg
"{""Id"":429,""Title"":""The Kissing Booth"",""Overview"":""When teenager Elle's first kiss leads to a forbidden romance with the hottest boy in high school, she risks her relationship with her best friend."",""Tagline"":""She can tell her best friend anything, except this one thing"",""Budget"":0.0,""Revenue"":0.0,""ImdbUrl"":""https://www.imdb.com/title/tt3799232"",""TmdbUrl"":""https://www.themoviedb.org/movie/454983"",""PosterUrl"":""https://image.tmdb.org/t/p/w342//7Dktk2ST6aL8h9Oe5rpk903VLhx.jpg"",""BackdropUrl"":""https://image.tmdb.org/t/p/original//itiz2OBK4ns6XT0ufXtusojmMt9.jpg"",""OriginalLanguage"":""en"",""ReleaseDate"":""2018-05-11T00:00:00"",""RunTime"":105,""Price"":9.9,""CreatedDate"":""2021-04-03T16:51:30.2366667"",""UpdatedDate"":null,""UpdatedBy"":null,""CreatedBy"":null,""genres"":[{""id"":7,""name"":""Comedy""},{""id"":16,""name"":""Romance""}]}",429,T

#### Step 2: Change all negative runtime to postive

In [0]:
from pyspark.sql.functions import when

bronzeQuarTransDF = bronzeQuarTransDF.withColumn('RunTime',abs(bronzeQuarTransDF.RunTime))
bronzeQuarTransDF = bronzeQuarTransDF.withColumn('Budget', when(bronzeQuarTransDF["Budget"] < 1000000, 1000000).otherwise(bronzeQuarTransDF["Budget"]))

In [0]:
display(bronzeQuarTransDF)

movie,Id,Title,Overview,OriginalLanguage,Price,ReleaseDate,p_ReleaseYear,Budget,Revenue,RunTime,Tagline,geres,CreatedBy,CreatedDate,UpdatedBy,UpdatedDate,ImdbUrl,TmdbUrl,PosterUrl,BackdropUrl
"{""Id"":323,""Title"":""Psycho"",""Overview"":""When larcenous real estate clerk Marion Crane goes on the lam with a wad of cash and hopes of starting a new life, she ends up at the notorious Bates Motel, where manager Norman Bates cares for his housebound mother."",""Tagline"":""A new—and altogether different—screen excitement!"",""Budget"":806947.0,""Revenue"":3.2E7,""ImdbUrl"":""https://www.imdb.com/title/tt0054215"",""TmdbUrl"":""https://www.themoviedb.org/movie/539"",""PosterUrl"":""https://image.tmdb.org/t/p/w342//nR4LD4ZJg2n6LZQpcOrLFdMq0cD.jpg"",""BackdropUrl"":""https://image.tmdb.org/t/p/original//kZGaVeXSkkvrpMYvD97sxHj291k.jpg"",""OriginalLanguage"":""en"",""ReleaseDate"":""1960-06-22T00:00:00"",""RunTime"":109,""Price"":9.9,""CreatedDate"":""2021-04-03T16:51:30.2200000"",""UpdatedDate"":null,""UpdatedBy"":null,""CreatedBy"":null,""genres"":[{""id"":4,""name"":""Drama""},{""id"":5,""name"":""Horror""},{""id"":10,""name"":""Thriller""}]}",323,Psycho,"When larcenous real estate clerk Marion Crane goes on the lam with a wad of cash and hopes of starting a new life, she ends up at the notorious Bates Motel, where manager Norman Bates cares for his housebound mother.",en,9.9,1960-06-22T00:00:00,1960,1000000.0,3.2E7,109,A new—and altogether different—screen excitement!,null,null,2021-04-03T16:51:30.2200000,null,null,https://www.imdb.com/title/tt0054215,https://www.themoviedb.org/movie/539,https://image.tmdb.org/t/p/w342//nR4LD4ZJg2n6LZQpcOrLFdMq0cD.jpg,https://image.tmdb.org/t/p/original//kZGaVeXSkkvrpMYvD97sxHj291k.jpg
"{""Id"":335,""Title"":""To All the Boys I've Loved Before"",""Overview"":""Lara Jean's love life goes from imaginary to out of control when her secret letters to every boy she's ever fallen for are mysteriously mailed out."",""Tagline"":""The Letters Are Out."",""Budget"":0.0,""Revenue"":0.0,""ImdbUrl"":""https://www.imdb.com/title/tt3846674"",""TmdbUrl"":""https://www.themoviedb.org/movie/466282"",""PosterUrl"":""https://image.tmdb.org/t/p/w342//hKHZhUbIyUAjcSrqJThFGYIR6kI.jpg"",""BackdropUrl"":""https://image.tmdb.org/t/p/original//hBHxBOGQBTMX3bDmqKoAgniZ9hE.jpg"",""OriginalLanguage"":""en"",""ReleaseDate"":""2018-08-16T00:00:00"",""RunTime"":100,""Price"":9.9,""CreatedDate"":""2021-04-03T16:51:30.2233333"",""UpdatedDate"":null,""UpdatedBy"":null,""CreatedBy"":null,""genres"":[{""id"":7,""name"":""Comedy""},{""id"":16,""name"":""Romance""}]}",335,To All the Boys I've Loved Before,Lara Jean's love life goes from imaginary to out of control when her secret letters to every boy she's ever fallen for are mysteriously mailed out.,en,9.9,2018-08-16T00:00:00,2018,1000000.0,0.0,100,The Letters Are Out.,null,null,2021-04-03T16:51:30.2233333,null,null,https://www.imdb.com/title/tt3846674,https://www.themoviedb.org/movie/466282,https://image.tmdb.org/t/p/w342//hKHZhUbIyUAjcSrqJThFGYIR6kI.jpg,https://image.tmdb.org/t/p/original//hBHxBOGQBTMX3bDmqKoAgniZ9hE.jpg
"{""Id"":429,""Title"":""The Kissing Booth"",""Overview"":""When teenager Elle's first kiss leads to a forbidden romance with the hottest boy in high school, she risks her relationship with her best friend."",""Tagline"":""She can tell her best friend anything, except this one thing"",""Budget"":0.0,""Revenue"":0.0,""ImdbUrl"":""https://www.imdb.com/title/tt3799232"",""TmdbUrl"":""https://www.themoviedb.org/movie/454983"",""PosterUrl"":""https://image.tmdb.org/t/p/w342//7Dktk2ST6aL8h9Oe5rpk903VLhx.jpg"",""BackdropUrl"":""https://image.tmdb.org/t/p/original//itiz2OBK4ns6XT0ufXtusojmMt9.jpg"",""OriginalLanguage"":""en"",""ReleaseDate"":""2018-05-11T00:00:00"",""RunTime"":105,""Price"":9.9,""CreatedDate"":""2021-04-03T16:51:30.2366667"",""UpdatedDate"":null,""UpdatedBy"":null,""CreatedBy"":null,""genres"":[{""id"":7,""name"":""Comedy""},{""id"":16,""name"":""Romance""}]}

#### Step 3: Select correct records

In [0]:
(
    bronzeQuarTransDF.select(
     'Id',
     'Title',
     'Overview',
     'OriginalLanguage',
     'Price',
     'ReleaseDate',
     'Budget',
     'Revenue',
     'RunTime',
     'Tagline',
     'geres',
     'CreatedBy',
     'CreatedDate',
     'UpdatedBy',
     'UpdatedDate',
     'ImdbUrl',
     'TmdbUrl',
     'PosterUrl',
     'BackdropUrl',
     'p_ReleaseYear'
    )
    .write.format("delta")
    .mode("append")
    .partitionBy("p_ReleaseYear")
    .save(silverPath)
)

In [0]:
#funtion
bronzePath = "/FileStore/movieBronze"

from delta.tables import DeltaTable

def update_bronze_table_status(
    spark: SparkSession, bronzeTablePath: str, dataframe: DataFrame, status: str
) -> bool:

    bronzeTable = DeltaTable.forPath(spark, bronzePath)
    dataframeAugmented = dataframe.withColumn("status", lit(status))

    update_match = "bronze.movie = dataframe.movie"
    update = {"status": "dataframe.status"}

    (
        bronzeTable.alias("bronze")
        .merge(dataframeAugmented.dropDuplicates().alias("dataframe"), update_match)
        .whenMatchedUpdate(set=update)
        .execute()
    )

    return True

update_bronze_table_status(spark, bronzePath, bronzeQuarTransDF, "loaded")

Out[17]: True

### Check all tables

In [0]:
%sql
SELECT count(id)
FROM silver_movie_silver;

count(id)
9995


In [0]:
%sql
SELECT count(movie)
FROM raw_movie_data_clissic_bronze;

count(movie)
9995
